<center> <h2>Using Py-ART to read and examine data from BoM Radars</h2> </center>
<center> Scott Collis, Argonne National Laboratory </center>

In [ ]:
# first some imports

import pyart # The Python ARM Radar Toolkit
import cartopy.crs as ccrs # A toolkit for map projections
import cartopy.feature as cfeature # A method for adding coastlines etc
import cartopy.io.img_tiles as cimgt # an API into things like google maps
import numpy as np #The famous numpy package!
from matplotlib import pyplot as plt
#Now a Jupyter "Magic" for plotting inline
%matplotlib inline


In [ ]:
radar = pyart.aux_io.read_odim_h5('../data/66_20181025_084830.pvol.h5')

In [ ]:
#lets see what is in the file
print("The radar has ", radar.ngates, " gates per ray")
print("The radar has ", radar.nrays, " rays per volume")
print("The radar has ", radar.nsweeps, " Sweeps per volume")
for i in range(len(radar.fixed_angle['data'])):
    sweep_start = radar.sweep_start_ray_index['data'][i]
    sweep_end = radar.sweep_start_ray_index['data'][i]
    nrays_in_sweep = sweep_end - sweep_start
    angle = radar.elevation['data'][sweep_start + int(nrays_in_sweep/2.)]
    print("Sweep ", i, " has an elevation of ", angle, ' degrees')




In [ ]:
#Radar data is in radar.fields
for field in radar.fields.keys():
    print(field, ' has units ', radar.fields[field]['units'])

### In Py-ART the data is always in data.. lets take a look!

In [ ]:
print(radar.fields['reflectivity']['data'])

In [ ]:
print('The mean reflectivity is ', radar.fields['reflectivity']['data'].mean())

In [ ]:
z_threshold =0.0

#where command gives you range, ray pairs where a condition is met
#so we take the first element (the time array) and get the lenghth of that 
n_gates_over = np.where(radar.fields['reflectivity']['data'] > z_threshold)[0].shape[0]

#Now take the total gates to get percentage
total_gates = radar.fields['reflectivity']['data'].shape[0] * radar.fields['reflectivity']['data'].shape[1]
percentage_over = 100.0 * n_gates_over/total_gates
print(percentage_over, '% of gates are over ', z_threshold)

In [ ]:
#lets look at the data via a "Bscan"
my_figure = plt.figure(figsize=[14,10]) 
plt.pcolormesh( radar.range['data'], radar.time['data'], radar.fields['reflectivity']['data'],
              cmap = pyart.graph.cm_colorblind.HomeyerRainbow, vmin=-8, vmax = 64)
plt.ylabel('Time (Seconds since scan start)')
plt.xlabel('Range (m)')

plt.colorbar()

In [ ]:
#woo!!! Data! Pretty damn ugly though.. Py-ART can help with transforms and plotting

In [ ]:
my_figure = plt.figure(figsize=[15,7])
my_radar_display = pyart.graph.RadarDisplay(radar)
my_radar_display.plot_ppi('reflectivity', 0, 
                          cmap=pyart.graph.cm_colorblind.HomeyerRainbow)

In [ ]:
#pretty! but no context!
#lets use the Cartopy package to plot on a map

#I get a bit geeky on this... 

# lets get the extent of the data

lats = radar.gate_latitude
lons = radar.gate_longitude

min_lon = lons['data'].min()
min_lat = lats['data'].min()
max_lat = lats['data'].max()
max_lon = lons['data'].max()

print('min_lat:', min_lat, ' min_lon:', min_lon, 
      ' max_lat:', max_lat, ' max_lon:', max_lon)


#set up the plot
myf = plt.figure(figsize=[20,18])
display = pyart.graph.RadarMapDisplayCartopy(radar)
lat_0 = display.loc[0]
lon_0 = display.loc[1]

# Set up the GIS projection
projection = ccrs.Mercator(
                central_longitude=lon_0,
                min_latitude=min_lat, max_latitude=max_lat)

#plot a PPI. We specifically ask Py-ART not to add the colorbar... 
#We will customize this to make it nice.

display.plot_ppi_map(
    'reflectivity', 0,
    projection=projection, colorbar_flag=False,
    min_lon=min_lon, max_lon=max_lon, min_lat=min_lat, max_lat=max_lat,
    vmin=-8, vmax=64, cmap=pyart.graph.cm_colorblind.HomeyerRainbow,
    resolution='10m')


#here is our pretty colorbar code
lb = display._get_colorbar_label('reflectivity')
cb = plt.colorbar(display.plots[0], aspect=30, pad=0.07, 
                  orientation='horizontal')

cb.ax.tick_params(labelsize=20)

cb.set_label(lb, fontsize=20)

#Now we add lat lon lines
gl = display.ax.gridlines(draw_labels=True,
                          linewidth=2, color='gray', alpha=0.5,
                          linestyle='--')

gl.xlabel_style = {'size': 20}
gl.ylabel_style = {'size': 20}

gl.xlabels_top = False
gl.ylabels_right = False


In [ ]:
#pretty! but no context!
#lets use the Cartopy package to plot on a map

#I get a bit geeky on this... 

# lets get the extent of the data

lats = radar.gate_latitude
lons = radar.gate_longitude

min_lon = lons['data'].min()
min_lat = lats['data'].min()
max_lat = lats['data'].max()
max_lon = lons['data'].max()

print('min_lat:', min_lat, ' min_lon:', min_lon, 
      ' max_lat:', max_lat, ' max_lon:', max_lon)


#set up the plot
myf = plt.figure(figsize=[20,18])
display = pyart.graph.RadarMapDisplayCartopy(radar)
lat_0 = display.loc[0]
lon_0 = display.loc[1]

# Set up the GIS projection
projection = ccrs.Mercator(
                central_longitude=lon_0,
                min_latitude=min_lat, max_latitude=max_lat)

#plot a PPI. We specifically ask Py-ART not to add the colorbar... 
#We will customize this to make it nice.

display.plot_ppi_map(
    'differential_reflectivity', 0,
    projection=projection, colorbar_flag=False,
    min_lon=min_lon, max_lon=max_lon, min_lat=min_lat, max_lat=max_lat,
    vmin=-.2, vmax=6, cmap=pyart.graph.cm_colorblind.HomeyerRainbow,
    resolution='10m')


#here is our pretty colorbar code
lb = display._get_colorbar_label('reflectivity')
cb = plt.colorbar(display.plots[0], aspect=30, pad=0.07, 
                  orientation='horizontal')

cb.ax.tick_params(labelsize=20)

cb.set_label(lb, fontsize=20)

#Now we add lat lon lines
gl = display.ax.gridlines(draw_labels=True,
                          linewidth=2, color='gray', alpha=0.5,
                          linestyle='--')

gl.xlabel_style = {'size': 20}
gl.ylabel_style = {'size': 20}

gl.xlabels_top = False
gl.ylabels_right = False


## Lets have a simple look at Py-ART's gatefilter

In [ ]:
#first.. Lets get rid of some things we do not like

my_gf = pyart.filters.GateFilter(radar)
my_gf.exclude_above('differential_reflectivity', 6)
my_gf.exclude_below('cross_correlation_ratio', .8)
my_ds_gf = pyart.correct.despeckle_field(radar, 'reflectivity', gatefilter=my_gf)

In [ ]:
#pretty! but no context!
#lets use the Cartopy package to plot on a map

#I get a bit geeky on this... 

# lets get the extent of the data

how_far_south = 0.2

lats = radar.gate_latitude
lons = radar.gate_longitude

min_lon = lons['data'].min()
min_lat = lats['data'].min()
max_lat = lats['data'].max()
max_lon = lons['data'].max()

print('min_lat:', min_lat, ' min_lon:', min_lon, 
      ' max_lat:', max_lat, ' max_lon:', max_lon)


#set up the plot
myf = plt.figure(figsize=[20,18])
display = pyart.graph.RadarMapDisplayCartopy(radar)
lat_0 = display.loc[0]
lon_0 = display.loc[1]

# Set up the GIS projection
projection = ccrs.Mercator(
                central_longitude=lon_0,
                min_latitude=min_lat-how_far_south, max_latitude=max_lat)

#plot a PPI. We specifically ask Py-ART not to add the colorbar... 
#We will customize this to make it nice.

display.plot_ppi_map(
    'reflectivity', 0,
    projection=projection, colorbar_flag=False,
    vmin=-8, vmax=64, cmap=pyart.graph.cm_colorblind.HomeyerRainbow, gatefilter=my_ds_gf,
    resolution='10m')

display.ax.set_extent([151, 154, -29, -27])

#here is our pretty colorbar code
lb = display._get_colorbar_label('reflectivity')
cb = plt.colorbar(display.plots[0], aspect=30, pad=0.07, 
                  orientation='horizontal')

request = cimgt.StamenTerrain()
display.ax.add_image(request, 8, zorder=0)

#adapted from https://stackoverflow.com/questions/25416600/why-the-annotate-worked-unexpected-here-in-cartopy
transform = ccrs.PlateCarree()._as_mpl_transform(display.ax)

display.ax.annotate('High Reflectivity', xy=(152.6, -27.8), xytext=(152.0, -27.8),
            arrowprops=dict(facecolor='gray',
                            arrowstyle="simple",
                            connectionstyle="arc3,rad=-0.2"),
            xycoords=transform,
            ha='right', va='top', fontsize=20)
  
    
cb.ax.tick_params(labelsize=20)

cb.set_label(lb, fontsize=20)

#Now we add lat lon lines
gl = display.ax.gridlines(draw_labels=True,
                          linewidth=2, color='gray', alpha=0.5,
                          linestyle='--')

gl.xlabel_style = {'size': 20}
gl.ylabel_style = {'size': 20}

gl.xlabels_top = False
gl.ylabels_right = False


In [ ]:
#pretty! but no context!
#lets use the Cartopy package to plot on a map

#I get a bit geeky on this... 

# lets get the extent of the data

how_far_south = 0.2

lats = radar.gate_latitude
lons = radar.gate_longitude

min_lon = lons['data'].min()
min_lat = lats['data'].min()
max_lat = lats['data'].max()
max_lon = lons['data'].max()

print('min_lat:', min_lat, ' min_lon:', min_lon, 
      ' max_lat:', max_lat, ' max_lon:', max_lon)


#set up the plot
myf = plt.figure(figsize=[20,18])
display = pyart.graph.RadarMapDisplayCartopy(radar)
lat_0 = display.loc[0]
lon_0 = display.loc[1]

# Set up the GIS projection
projection = ccrs.Mercator(
                central_longitude=lon_0,
                min_latitude=min_lat-how_far_south, max_latitude=max_lat)

#plot a PPI. We specifically ask Py-ART not to add the colorbar... 
#We will customize this to make it nice.

display.plot_ppi_map(
    'differential_reflectivity', 0,
    projection=projection, colorbar_flag=False,
    vmin=.1, vmax=6, cmap=pyart.graph.cm_colorblind.HomeyerRainbow, gatefilter=my_ds_gf,
    resolution='10m')

display.ax.set_extent([151, 154, -29, -27])

#here is our pretty colorbar code
lb = display._get_colorbar_label('reflectivity')
cb = plt.colorbar(display.plots[0], aspect=30, pad=0.07, 
                  orientation='horizontal')

request = cimgt.StamenTerrain()
display.ax.add_image(request, 8, zorder=0)


#adapted from https://stackoverflow.com/questions/25416600/why-the-annotate-worked-unexpected-here-in-cartopy
transform = ccrs.PlateCarree()._as_mpl_transform(display.ax)

display.ax.annotate('High Reflectivity', xy=(152.6, -27.8), xytext=(152.0, -27.8),
            arrowprops=dict(facecolor='gray',
                            arrowstyle="simple",
                            connectionstyle="arc3,rad=-0.2"),
            xycoords=transform,
            ha='right', va='top', fontsize=20)
    
cb.ax.tick_params(labelsize=20)

cb.set_label(lb, fontsize=20)

#Now we add lat lon lines
gl = display.ax.gridlines(draw_labels=True,
                          linewidth=2, color='gray', alpha=0.5,
                          linestyle='--')

gl.xlabel_style = {'size': 20}
gl.ylabel_style = {'size': 20}

gl.xlabels_top = False
gl.ylabels_right = False



In [ ]:
#now for someting quantitative.. WARNING.. NEXRAD Z-R dont use for MtSt :)
rain_z = radar.fields['reflectivity']['data'].copy()
z_lin = 10.0**(radar.fields['reflectivity']['data']/10.)
rain_z = (z_lin/300.0)**(1./1.4)  #Z=300 R1.4
radar.add_field_like('reflectivity', 'rain_z',  rain_z, replace_existing = True)
radar.fields['rain_z']['units'] = 'mm/h'
radar.fields['rain_z']['standard_name'] = 'rainfall_rate'
radar.fields['rain_z']['long_name'] = 'rainfall_rate_from_z'
radar.fields['rain_z']['valid_min'] = 0
radar.fields['rain_z']['valid_max'] = 500


In [ ]:
#pretty! but no context!
#lets use the Cartopy package to plot on a map

#I get a bit geeky on this... 

# lets get the extent of the data

how_far_south = 0.3

lats = radar.gate_latitude
lons = radar.gate_longitude

min_lon = lons['data'].min()
min_lat = lats['data'].min()
max_lat = lats['data'].max()
max_lon = lons['data'].max()

print('min_lat:', min_lat, ' min_lon:', min_lon, 
      ' max_lat:', max_lat, ' max_lon:', max_lon)


#set up the plot
myf = plt.figure(figsize=[20,18])
display = pyart.graph.RadarMapDisplayCartopy(radar)
lat_0 = display.loc[0]
lon_0 = display.loc[1]

# Set up the GIS projection
projection = ccrs.Mercator(
                central_longitude=lon_0,
                min_latitude=min_lat-how_far_south, max_latitude=max_lat)

#plot a PPI. We specifically ask Py-ART not to add the colorbar... 
#We will customize this to make it nice.

display.plot_ppi_map(
    'rain_z', 0,
    projection=projection, colorbar_flag=False,
    vmin=0, vmax=150, cmap=pyart.graph.cm_colorblind.HomeyerRainbow, gatefilter=my_ds_gf,
    resolution='10m')

display.ax.set_extent([151, 154, -29, -27])

#here is our pretty colorbar code
lb = display._get_colorbar_label('rain_z')
cb = plt.colorbar(display.plots[0], aspect=30, pad=0.07, 
                  orientation='horizontal')

request = cimgt.StamenTerrain()
display.ax.add_image(request, 8, zorder=0)

transform = ccrs.PlateCarree()._as_mpl_transform(display.ax)

display.ax.annotate('Hail contamination', xy=(152.6, -27.8), xytext=(152.0, -27.8),
            arrowprops=dict(facecolor='gray',
                            arrowstyle="simple",
                            connectionstyle="arc3,rad=-0.2"),
            xycoords=transform,
            ha='right', va='top', fontsize=20)

    
cb.ax.tick_params(labelsize=20)

cb.set_label(lb, fontsize=20)

#Now we add lat lon lines
gl = display.ax.gridlines(draw_labels=True,
                          linewidth=2, color='gray', alpha=0.5,
                          linestyle='--')

gl.xlabel_style = {'size': 20}
gl.ylabel_style = {'size': 20}

gl.xlabels_top = False
gl.ylabels_right = False
